# Dockerizing Lab

In this lab, you'll learn how to containerize a Python application using Docker. By the end, you'll have a working container that you can run anywhere Docker is installed.

**What you'll learn:**
- Writing a Dockerfile
- Building a Docker image
- Running and managing containers
- Using Docker Compose

**Prerequisites:**
- Docker Desktop installed and running (`docker --version`)
- Basic understanding of the terminal
- Python basics (helpful but not required)

**Time:** ~30-40 minutes

---

## Part 1: The Demo Application

We've provided a simple Python web application in the `demo-app/` folder. Let's look at what it contains:

```
demo-app/
├── app.py              # The main application
├── requirements.txt    # Python dependencies
├── Dockerfile          # Instructions to build the image
└── docker-compose.yml  # Multi-container configuration
```

### The Application (`app.py`)

It's a simple web server using FastAPI that returns a greeting:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello from Docker!"}

@app.get("/greet/{name}")
def greet(name: str):
    return {"message": f"Hello, {name}!"}
```

### Navigate to the demo app

Open your terminal and navigate to the `demo-app` folder:

```bash
cd demo-app
```

---

## Part 2: Running Without Docker (The Old Way)

First, let's see how you'd normally run this app. This helps you appreciate what Docker does for you.

### Step 1: Create a virtual environment

```bash
python -m venv venv
```

### Step 2: Activate it

```bash
# Windows
venv\Scripts\activate

# macOS/Linux
source venv/bin/activate
```

### Step 3: Install dependencies

```bash
pip install -r requirements.txt
```

### Step 4: Run the app

```bash
uvicorn app:app --host 0.0.0.0 --port 8000
```

Open your browser to `http://localhost:8000` — you should see `{"message": "Hello from Docker!"}`

**Stop the server** with `Ctrl+C` and deactivate the virtual environment:

```bash
deactivate
```

That was 4 steps. With Docker, it's just one command. Let's see how.

---

## Part 3: Understanding the Dockerfile

A **Dockerfile** is a recipe for building an image. Here's the one for our app:

```dockerfile
# Start from an official Python image
FROM python:3.11-slim

# Set the working directory inside the container
WORKDIR /app

# Copy requirements first (for better caching)
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code
COPY app.py .

# Document which port the app uses
EXPOSE 8000

# Command to run when the container starts
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

### Line by line:

| Line | What it does |
|------|-------------|
| `FROM python:3.11-slim` | Start with a pre-built Python image (slim = smaller size) |
| `WORKDIR /app` | All following commands happen in `/app` |
| `COPY requirements.txt .` | Copy requirements file into the image |
| `RUN pip install ...` | Install Python packages |
| `COPY app.py .` | Copy our application code |
| `EXPOSE 8000` | Document that the app uses port 8000 |
| `CMD [...]` | The default command to run |

### Why copy requirements.txt separately?

Docker caches each layer. If `requirements.txt` hasn't changed, Docker reuses the cached layer with installed packages. This makes rebuilds much faster when you only change `app.py`.

---

## Part 4: Building the Image

Now let's build our Docker image.

### Make sure you're in the demo-app folder

```bash
cd demo-app
```

### Build the image

```bash
docker build -t demo-app .
```

Let's break this down:
- `docker build` — Build an image from a Dockerfile
- `-t demo-app` — Tag (name) the image as "demo-app"
- `.` — Use the current directory as the build context

You'll see Docker executing each step in the Dockerfile. The first build takes longer because it downloads the Python base image.

### Verify the image was created

```bash
docker images
```

You should see `demo-app` in the list.

---

## Part 5: Running the Container

An image is just a template. To actually run the app, we create a **container** from the image.

### Run the container

```bash
docker run -p 8000:8000 demo-app
```

- `docker run` — Create and start a container
- `-p 8000:8000` — Map port 8000 on your machine to port 8000 in the container
- `demo-app` — The image to use

Open your browser to `http://localhost:8000` — same result as before, but now it's running in a container!

Try the greeting endpoint: `http://localhost:8000/greet/Docker`

**Stop the container** with `Ctrl+C`.

### Run in the background (detached mode)

```bash
docker run -d -p 8000:8000 --name my-app demo-app
```

- `-d` — Run in detached mode (background)
- `--name my-app` — Give the container a name

The app is now running in the background. Verify:

```bash
docker ps
```

### View logs

```bash
docker logs my-app
```

### Stop the container

```bash
docker stop my-app
```

### Remove the container

```bash
docker rm my-app
```

Or stop and remove in one command:

```bash
docker rm -f my-app
```

---

## Part 6: Useful Container Commands

Let's explore some common operations. First, start a container:

```bash
docker run -d -p 8000:8000 --name my-app demo-app
```

### Execute a command inside the container

```bash
docker exec my-app ls /app
```

This runs `ls /app` inside the container and shows the output.

### Open a shell inside the container

```bash
docker exec -it my-app bash
```

Now you're "inside" the container. Look around:

```bash
pwd
ls
python --version
pip list
exit
```

### View resource usage

```bash
docker stats
```

Press `Ctrl+C` to exit.

### Clean up

```bash
docker stop my-app
docker rm my-app
```

---

## Part 7: Docker Compose

For real applications, you often have multiple services (web server, database, cache, etc.). **Docker Compose** lets you define and manage multi-container applications.

### The docker-compose.yml file

```yaml
version: '3.8'

services:
  web:
    build: .
    ports:
      - "8000:8000"
    volumes:
      - .:/app
    environment:
      - DEBUG=true
```

This defines a single service called `web` that:
- Builds from the Dockerfile in the current directory
- Maps port 8000
- Mounts the current directory to `/app` (for live code reloading)
- Sets an environment variable

### Start with Docker Compose

```bash
docker compose up
```

Or in the background:

```bash
docker compose up -d
```

### View logs

```bash
docker compose logs
```

### Stop everything

```bash
docker compose down
```

### Rebuild after code changes

```bash
docker compose up --build
```

---

## Part 8: Making Changes

Let's see what happens when you modify the application.

### With volume mounting (Docker Compose)

Start the app with Docker Compose:

```bash
docker compose up
```

Now edit `app.py` and change the message:

```python
@app.get("/")
def read_root():
    return {"message": "Hello from Docker! I've been updated!"}
```

Because we mounted the local directory as a volume, you might see changes immediately (depending on the framework's hot-reload capability). If not, restart:

```bash
docker compose restart
```

### Without volume mounting

If you run with just `docker run`, you need to rebuild the image:

```bash
docker build -t demo-app .
docker run -p 8000:8000 demo-app
```

---

## Part 9: Writing Your Own Dockerfile

Now that you understand the concepts, try writing a Dockerfile from scratch.

### Exercise: Containerize a simple script

1. Create a new folder:

```bash
mkdir my-docker-app
cd my-docker-app
```

2. Create a simple Python script (`main.py`):

```python
import time

print("Starting application...")
for i in range(5):
    print(f"Tick {i + 1}")
    time.sleep(1)
print("Done!")
```

3. Write a Dockerfile:

```dockerfile
# Your turn! Fill this in:
# - Start from python:3.11-slim
# - Set workdir to /app
# - Copy main.py
# - Set the command to run python main.py
```

4. Build and run:

```bash
docker build -t my-docker-app .
docker run my-docker-app
```

You should see the ticks printed, then "Done!"

<details>
<summary>Click for solution</summary>

```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY main.py .
CMD ["python", "main.py"]
```

</details>

---

## Summary

| Concept | Description |
|---------|-------------|
| **Image** | A blueprint for containers (built from Dockerfile) |
| **Container** | A running instance of an image |
| **Dockerfile** | Instructions to build an image |
| **Docker Compose** | Tool for multi-container applications |

### Key Commands

| Command | What it does |
|---------|-------------|
| `docker build -t name .` | Build image from Dockerfile |
| `docker run -p 8000:8000 image` | Run container with port mapping |
| `docker run -d image` | Run in background |
| `docker ps` | List running containers |
| `docker stop name` | Stop a container |
| `docker logs name` | View container logs |
| `docker exec -it name bash` | Shell into container |
| `docker compose up` | Start compose services |
| `docker compose down` | Stop compose services |

---

## Exercises

1. **Modify the app**: Add a new endpoint to `app.py` (e.g., `/health` that returns `{"status": "ok"}`), rebuild, and test it

2. **Explore images**: Pull and run `nginx` (`docker run -p 80:80 nginx`) and visit `http://localhost`

3. **Interactive Python**: Run `docker run -it python:3.11` to get an interactive Python shell inside a container

4. **Environment variables**: Modify the app to read a `GREETING` environment variable and use it in the response. Pass it with `docker run -e GREETING=Hi ...`

---

## Next Section

Ready to build APIs? Continue to: **[APIs and Backend Development](../03-apis-and-backend/)**